## Column

A column in a DataFrame.

### Import Required modules and initialize SparkSession

In [1]:
from __future__ import print_function
import findspark
findspark.init()
import pyspark
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName("DataFrame Functions").getOrCreate()

In [3]:
spark

### Create DataFrame with sample data

In [4]:
from pyspark.sql import Row

data = [
    Row(
        first_name="Satish", second_name="Kumar", age=35, experience=12, salary=80000., 
        assets=Row(laptop=1, desktop=1),
        phone_nums=['123456', '222211'],
        others={"has_vehical": "yes"}
    ),
    Row(
        first_name="Ramya", second_name="Sree", age=30, experience=8, salary=50000.,
        assets=Row(laptop=1, desktop=0),
        phone_nums=['123457', '222212'],
        others={"has_vehical": "yes"}
    ),
    Row(
        first_name="Tejaswini", second_name="Uppara", age=None, experience=3, salary=30000., 
        assets=Row(laptop=0,desktop=1),
        phone_nums=['222213'],
        others={"has_vehical": "no"}
    ),
    Row(
        first_name="Bhavishya", second_name="Uppara", age=None, experience=2, salary=20000., 
        assets=Row(laptop=0,desktop=1),
        phone_nums=['222213'],
        others={"has_vehical": "no"}
    )
]

df = spark.createDataFrame(data)

In [5]:
df.show(truncate = False)

+----------+-----------+----+----------+-------+------+----------------+--------------------+
|first_name|second_name|age |experience|salary |assets|phone_nums      |others              |
+----------+-----------+----+----------+-------+------+----------------+--------------------+
|Satish    |Kumar      |35  |12        |80000.0|{1, 1}|[123456, 222211]|{has_vehical -> yes}|
|Ramya     |Sree       |30  |8         |50000.0|{1, 0}|[123457, 222212]|{has_vehical -> yes}|
|Tejaswini |Uppara     |null|3         |30000.0|{0, 1}|[222213]        |{has_vehical -> no} |
|Bhavishya |Uppara     |null|2         |20000.0|{0, 1}|[222213]        |{has_vehical -> no} |
+----------+-----------+----+----------+-------+------+----------------+--------------------+



### Explore Column API

In [6]:
df.first_name

Column<'first_name'>

In [7]:
df['salary']

Column<'salary'>

In [8]:
df.salary * 0.2

Column<'(salary * 0.2)'>

In [9]:
df.select(df.first_name, df['salary'], df.salary * 0.2).show(truncate=False)

+----------+-------+--------------+
|first_name|salary |(salary * 0.2)|
+----------+-------+--------------+
|Satish    |80000.0|16000.0       |
|Ramya     |50000.0|10000.0       |
|Tejaswini |30000.0|6000.0        |
|Bhavishya |20000.0|4000.0        |
+----------+-------+--------------+



#### Column aliased with a new name using 'alias' 

In [10]:
df.select(df.first_name, df['salary'], (df.salary * 0.2).alias('Salary 20 percent')).show(truncate=False)

+----------+-------+-----------------+
|first_name|salary |Salary 20 percent|
+----------+-------+-----------------+
|Satish    |80000.0|16000.0          |
|Ramya     |50000.0|10000.0          |
|Tejaswini |30000.0|6000.0           |
|Bhavishya |20000.0|4000.0           |
+----------+-------+-----------------+



In [11]:
df.select(df.experience.alias("experience1")).show()

+-----------+
|experience1|
+-----------+
|         12|
|          8|
|          3|
|          2|
+-----------+



In [12]:
df.select(df.experience.name("experience1")).show()

+-----------+
|experience1|
+-----------+
|         12|
|          8|
|          3|
|          2|
+-----------+



In [13]:
df.select(df.experience.alias("experience1", metadata={'min': 1})).schema['experience1'].metadata['min']

1

Optional argument 'metadata' – a dict of information to be stored in metadata attribute of the corresponding StructField class

### Get field by name using 'getField(name)'

In [14]:
df.select(df.assets).show()

+------+
|assets|
+------+
|{1, 1}|
|{1, 0}|
|{0, 1}|
|{0, 1}|
+------+



In [15]:
(
    df.select(
        df.assets.getField('laptop').name('has_laptop'),
        df.assets.getField('desktop').name('has_desktop'),
    )
).show()

+----------+-----------+
|has_laptop|has_desktop|
+----------+-----------+
|         1|          1|
|         1|          0|
|         0|          1|
|         0|          1|
+----------+-----------+



In [16]:
(
    df.select(
        df.assets.laptop.name('has_laptop'),
        df.assets.desktop.name('has_desktop'),
    )
).show()

+----------+-----------+
|has_laptop|has_desktop|
+----------+-----------+
|         1|          1|
|         1|          0|
|         0|          1|
|         0|          1|
+----------+-----------+



### Get an item at position ordinal out of a list, or gets an item by key out of a dict using 'getItem(key)'

In [17]:
df.select(df.phone_nums,df.others).show()

+----------------+--------------------+
|      phone_nums|              others|
+----------------+--------------------+
|[123456, 222211]|{has_vehical -> yes}|
|[123457, 222212]|{has_vehical -> yes}|
|        [222213]| {has_vehical -> no}|
|        [222213]| {has_vehical -> no}|
+----------------+--------------------+



In [18]:
df.select(df.phone_nums.getItem(0),df.others.getItem('has_vehical')).show()

+-------------+-------------------+
|phone_nums[0]|others[has_vehical]|
+-------------+-------------------+
|       123456|                yes|
|       123457|                yes|
|       222213|                 no|
|       222213|                 no|
+-------------+-------------------+



### Convert the column dataType

In [19]:
df1 = df.select(df.age, df.age.cast("string").alias('age_str'))

In [20]:
df1.printSchema()

root
 |-- age: long (nullable = true)
 |-- age_str: string (nullable = true)



In [21]:
df1.show()

+----+-------+
| age|age_str|
+----+-------+
|  35|     35|
|  30|     30|
|null|   null|
|null|   null|
+----+-------+



### Filter the columns with Null and Non-Null values 

In [22]:
df.show(truncate=False)

+----------+-----------+----+----------+-------+------+----------------+--------------------+
|first_name|second_name|age |experience|salary |assets|phone_nums      |others              |
+----------+-----------+----+----------+-------+------+----------------+--------------------+
|Satish    |Kumar      |35  |12        |80000.0|{1, 1}|[123456, 222211]|{has_vehical -> yes}|
|Ramya     |Sree       |30  |8         |50000.0|{1, 0}|[123457, 222212]|{has_vehical -> yes}|
|Tejaswini |Uppara     |null|3         |30000.0|{0, 1}|[222213]        |{has_vehical -> no} |
|Bhavishya |Uppara     |null|2         |20000.0|{0, 1}|[222213]        |{has_vehical -> no} |
+----------+-----------+----+----------+-------+------+----------------+--------------------+



In [23]:
# Get the records having age as Null
df.filter(df.age.isNull()).show(truncate=False)

+----------+-----------+----+----------+-------+------+----------+-------------------+
|first_name|second_name|age |experience|salary |assets|phone_nums|others             |
+----------+-----------+----+----------+-------+------+----------+-------------------+
|Tejaswini |Uppara     |null|3         |30000.0|{0, 1}|[222213]  |{has_vehical -> no}|
|Bhavishya |Uppara     |null|2         |20000.0|{0, 1}|[222213]  |{has_vehical -> no}|
+----------+-----------+----+----------+-------+------+----------+-------------------+



In [24]:
# Get the records having age is not Null
df.filter(df.age.isNotNull()).show(truncate=False)

+----------+-----------+---+----------+-------+------+----------------+--------------------+
|first_name|second_name|age|experience|salary |assets|phone_nums      |others              |
+----------+-----------+---+----------+-------+------+----------------+--------------------+
|Satish    |Kumar      |35 |12        |80000.0|{1, 1}|[123456, 222211]|{has_vehical -> yes}|
|Ramya     |Sree       |30 |8         |50000.0|{1, 0}|[123457, 222212]|{has_vehical -> yes}|
+----------+-----------+---+----------+-------+------+----------------+--------------------+



In [25]:
# Equality test that is safe for null values.
df.select(
    df.first_name,
    df.age,
    df['age'].eqNullSafe(None),
    df['age'].eqNullSafe(30)
).show()

+----------+----+--------------+------------+
|first_name| age|(age <=> NULL)|(age <=> 30)|
+----------+----+--------------+------------+
|    Satish|  35|         false|       false|
|     Ramya|  30|         false|        true|
| Tejaswini|null|          true|       false|
| Bhavishya|null|          true|       false|
+----------+----+--------------+------------+



### Get the substring of the column value

In [26]:
df.select(df.first_name, df.first_name.substr(1, 3).alias("first_name_3_let")).show()

+----------+----------------+
|first_name|first_name_3_let|
+----------+----------------+
|    Satish|             Sat|
|     Ramya|             Ram|
| Tejaswini|             Tej|
| Bhavishya|             Bha|
+----------+----------------+

